# This is a notebook using QAChain with trunks of graph stored in the Pinecone to communicate with the database

In [1]:
import pinecone
import openai
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase

from langchain.embeddings.base import Embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()
neo4j_url = os.getenv('NEO4J_URL')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')
openai_key = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = openai_key
pinecone_api_key = os.getenv('Pinecone_KEY')

embeddings = OpenAIEmbeddings()


/Users/rc/miniforge3/envs/chat/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Divide graph into trunks

In [2]:
# Setting of Neo4j
driver = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

# Divide the graph into trunks
def get_pair_nodes (tx):
    pairs_of_nodes = []
    for record in tx.run("MATCH (a)-[r]->(b) RETURN labels(a), a.name, type(r), labels(b), b.name"):
        pair_node = {
            "start_node_name": record["a.name"],
            "start_node_label":record["labels(a)"][0],
            "end_node_name": record["b.name"],
            "end_node_label":record["labels(b)"][0],
            "edge": record["type(r)"]
        }
        pairs_of_nodes.append(pair_node)
        # print(record["labels(a)"][0], record["a.name"], record["type(r)"], record["labels(b)"][0], record["b.name"])
    return pairs_of_nodes

with driver.session() as session:
    pairs_of_nodes = session.execute_read(get_pair_nodes)
driver.close()
pairs_of_nodes

[{'start_node_name': 'Review_project_portfolio_balance_for_2020_Q1',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'Portfolio_management',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Conduct_project_assurance_readiness_for_Brexit_Programme',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Assessment',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Identify_best_practice_regulatory_project_as_examplar',
  'start_node_label': 'Areas_of_support',
  'end_node_name': 'P3M_Support',
  'end_node_label': 'Our_Service_Offerings',
  'edge': 'deploy'},
 {'start_node_name': 'Cross-project_review_of_plastic_dependencies',
  'start_node_label': 'Areas_of_support',


In [ ]:
node_vectors = []
for pair_node in pairs_of_nodes:
    start_node_name = pair_node["start_node_name"]
    edge = pair_node["edge"]
    end_node_name = pair_node["end_node_name"]
    start_node_label = pair_node["start_node_label"]
    end_node_label = pair_node["end_node_label"]
    pair_node["context"] = f"{start_node_name} is {edge} {end_node_name}. {start_node_name} is one type of {start_node_label}. {end_node_name} is one type of {end_node_label}."
    pair_node["source"] = "ps-graph"
    node_per_pair = {
        "value": embeddings.embed_documents([pair_node["context"]]),
        "meta_data": pair_node,
    } 
    node_vectors.append(node_per_pair)
node_vectors

# Upload trunks of graph to Pinecone

In [ ]:
# Setting of pinecone
env_name = "us-west4-gcp-free"
index_name = "test-chatbot-ran"
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)

# upload vectors 
my_namespace = 'graph_02'
index.delete(deleteAll='true', namespace=my_namespace)
pinecone_vectors = []
for idx, n_v in enumerate(node_vectors):
    pinecone_vectors.append((str(idx), n_v["value"], n_v["meta_data"]))
upsert_response = index.upsert(vectors=pinecone_vectors, namespace=my_namespace)

# Response with Pinecone API

In [ ]:
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)

prompt = "what are the values of unilever create?" # vector created with OpenAI as well
query_vector = embeddings.embed_query(prompt)
response = index.query(
    top_k=10,
    vector=query_vector,
    include_metadata=True,
    namespace=my_namespace,
)
response

<font size=4>As we can see, the response return the similar vector from the prompt when using Pinecone API

# Use Langchain to search

# Option 1. Query using similarity search

In [ ]:
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

# The environment should be the one specified next to the API key
# in your Pinecone console
pinecone.init(api_key=pinecone_api_key,environment=env_name)
index = pinecone.Index(index_name)
vectorstore = Pinecone(index, embeddings.embed_query, "context", namespace=my_namespace)


query = "what are the values of unilever create?"
docs = vectorstore.similarity_search(query)
docs

# Option 2. Use chain to qurey

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "what are the values of unilever create?"
docs = vectorstore.similarity_search(query, k=10)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "how many types of migitigation associated with project risks and give their name please"
docs = vectorstore.similarity_search(query, k=10)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "How many types of Strategy_themes and give the list of their names?"
docs = vectorstore.similarity_search(query, k=10)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What are the names of Mitigation associated with Poor_project_execution"
docs = vectorstore.similarity_search(query, k=10)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "How many types of areas of support and give the list of their names?"
docs = vectorstore.similarity_search(query, k=10)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

# QA-Chain with Source returns reasonable output. However, it still depends on the top K search. BUT the result looks pretty good.